# Import Dataset from DB

In [ ]:
pwd

In [ ]:
import os
import pathlib
import sys
sys.path.append("..")

current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
output_dir = pathlib.Path('{}/output_220222/'.format(current_dir))
pathlib.Path.mkdir(output_dir, mode=0o777, parents=True, exist_ok=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
import os
import pandas as pd
import numpy as np
#!pip install pymssql
import pymssql
from datetime import timedelta

pd.set_option('display.max_colwidth', -1)  #각 컬럼 width 최대로 
pd.set_option('display.max_rows', 50)      # display 50개 까지 

# connection DataBase (해당 DB의 명령문 사용)

### Postgresql (계정유출 주의)

In [ ]:
# import pandas.io.sql as psql
# import psycogp2 as pg
# conn = pg.connect(host='111.1.11.11', dbname='cdm', user='1111', password='1111', port='5432')
# print('connection')

# sql = 'select * from cdm.cdm.person'
# population = psql.read(sql=sql1, con=conn)
# population.to_csv('population.txt', index=False)

### Mssql (계정유출 주의)

In [ ]:
import pymssql
#conn= pymssql.connect(server="111.111.111.111", user="user", password="password", port=22, database='db', as_dict=False)
print("connection")

### Table Load from DB

In [ ]:
tnPopulation = 'test.dbo.celecoxib_1'
tnMeasurement = 'CDM.dbo.measurement'
tnDrug = 'CDM.dbo.drug_exposure'
tnProcedure = 'CDM.dbo.procedure_occurrence'
tnCondition = 'CDM.dbo.condition_occurrence'
tnConcept = 'CDM.dbo.concept'


In [ ]:
# 간독성 Patient 불러오기

sqlq0 = 'select * from {}'.format(tnPopulation)
population_df = pd.read_sql(sql=sqlq0, con=conn)

sqlq1 ="select person_id, measurement_concept_id, measurement_date, value_as_number from {} ".format(tnMeasurement) +\
    "where measurement_concept_id!=0 and value_as_number is not null and person_id in (select distinct person_id from {})".format(tnPopulation)
measurement_df = pd.read_sql(sql=sqlq1, con=conn)

sqlq2="select person_id, drug_concept_id, drug_exposure_start_date, quantity from {} ".format(tnDrug) +\
    "where drug_concept_id!=0 and quantity is not null and person_id in (select distinct person_id from {})".format(tnPopulation)
drug_df = pd.read_sql(sql=sqlq2, con=conn)

sqlq3="select person_id, procedure_concept_id, procedure_date from {} ".format(tnProcedure) +\
    "where procedure_concept_id!=0 and person_id in (select distinct person_id from {} )".format(tnPopulation)
procedure_df = pd.read_sql(sql=sqlq3, con=conn)

sqlq4="select person_id, condition_concept_id, condition_start_date from {} ".format(tnCondition) +\
    "where condition_concept_id!=0 and person_id in (select distinct person_id from {} )".format(tnPopulation)
condition_df = pd.read_sql(sql=sqlq4, con=conn)

sqlq5="select distinct concept_id, concept_name from {}".format(tnConcept)
concept_df = pd.read_sql(sql=sqlq5, con=conn)

conn.close()

### Save dataset

In [ ]:
population_df.to_csv('{}/population.txt'.format(output_dir),index=False)
measurement_df.to_csv('{}/measurement.txt'.format(output_dir),index=False)
drug_df.to_csv('{}/drug.txt'.format(output_dir),index=False)
procedure_df.to_csv('{}/procedure.txt'.format(output_dir),index=False)
condition_df.to_csv('{}/condition.txt'.format(output_dir),index=False)
concept_df.to_csv('{}/concept.txt'.format(output_dir),index=False)

# Load dataset

In [ ]:
# 데이터 불러오기
population_df=pd.read_csv('{}/population.txt'.format(output_dir))
measurement_df=pd.read_csv('{}/measurement.txt'.format(output_dir))
drug_df=pd.read_csv('{}/drug.txt'.format(output_dir))
procedure_df=pd.read_csv('{}/procedure.txt'.format(output_dir))
condition_df=pd.read_csv('{}/condition.txt'.format(output_dir))
concept_df=pd.read_csv('{}/concept.txt'.format(output_dir))

In [ ]:
measurement_df = measurement_df[["person_id","measurement_concept_id","measurement_date","value_as_number"]]
drug_df = drug_df[["person_id","drug_concept_id","drug_exposure_start_date","quantity"]]
procedure_df = procedure_df[["person_id","procedure_concept_id","procedure_date"]]
condition_df = condition_df[["person_id","condition_concept_id","condition_start_date"]]
concept_df = concept_df[["concept_id","concept_name"]]

In [ ]:
# # 그룹바이 해서 10건 미만 리스트업
# c=measurement_df.columns
# gb=measurement_df[c].groupby(["measurement_concept_id"]).size().reset_index(name="cnt")
# gb_10=gb[gb["cnt"]<10]
# gb_df=pd.DataFrame(gb_10["measurement_concept_id"].unique())
# gb_df["measurement_concept_id"]=gb_df[0]
# # 해당 measurement_concept_id 를 포함하는 레코드 찾기
# idx=[]
# for i in gb_df["measurement_concept_id"]:    
#     d= measurement_df[measurement_df["measurement_concept_id"]==i]
#     d1=[i for i in d.index]    
#     idx.extend(d1)  
# # 인덱스로 제거 
# measurement_df2=measurement_df.drop(idx)
# measurement_df=measurement_df2.reset_index(drop=True)

# Only IO: generate each domain dataset

In [ ]:
# population for each domains
population_df=population_df[["subject_id","cohort_start_date","age","gender_source_value","person_id"]]
population_df=population_df.drop_duplicates()
len(population_df)
## measurement
IO_measurement = pd.merge(population_df, measurement_df, left_on=["person_id"], right_on=["person_id"], how="inner")
IO_measurement = pd.merge(IO_measurement, concept_df, left_on=["measurement_concept_id"],right_on=["concept_id"],how="inner")
IO_measurement = IO_measurement.reset_index(drop=True)
#drug
IO_drug = pd.merge(population_df, drug_df, left_on=["person_id"], right_on=["person_id"], how="inner")
IO_drug = pd.merge(IO_drug,concept_df,left_on=["drug_concept_id"],right_on=["concept_id"],how="inner")
IO_drug = IO_drug.reset_index(drop=True)
#procedure
IO_procedure = pd.merge(population_df, procedure_df, left_on=["person_id"], right_on=["person_id"], how="inner")
IO_procedure = pd.merge(IO_procedure,concept_df,left_on=["procedure_concept_id"],right_on=["concept_id"],how="inner")
IO_procedure = IO_procedure.reset_index(drop=True)
#condition
IO_condition = pd.merge(population_df, condition_df, left_on=["person_id"], right_on=["person_id"], how="inner")
IO_condition = pd.merge(IO_condition,concept_df,left_on=["condition_concept_id"],right_on=["concept_id"],how="inner")
IO_condition = IO_condition.reset_index(drop=True)

In [ ]:
del population_df
del measurement_df
del drug_df
del procedure_df
del condition_df
del concept_df

# CTCAE outcomes: generate each outcome dataset

# TARGETTING

In [ ]:
def cohortConditionSetting(domain_df, observation_period, interesting_period):
    from datetime import timedelta
    prev_len = len(domain_df)
    domain_df['cohort_start_date'] = pd.to_datetime(domain_df['cohort_start_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')
    domain_df['measurement_date'] = pd.to_datetime(domain_df['measurement_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')
    # condition 1) Select patients with first adverse events within 2 months of cohort initiation.
    domain_df = domain_df[(domain_df['cohort_start_date']-domain_df['measurement_date']<timedelta(days=observation_period))]
    # condition 2) Delete data before the cohort start date.
    domain_df = domain_df[(domain_df['measurement_date']-domain_df['cohort_start_date']<timedelta(days=interesting_period))]
    domain_df = domain_df.reset_index(drop=True)
    curr_len = len(domain_df)
    print('{} > {}'.format(prev_len, curr_len))
    return domain_df

IO_measurement = cohortConditionSetting(IO_measurement, observation_period=60, interesting_period=60)

In [ ]:
def measurement_targetting(abrnoaml_ctcae, IO_measurement):
    # ID, Measure_date 순 정렬
    abnormal_ctcae_idx = abrnoaml_ctcae.sort_values(by=["person_id","measurement_date"],ascending=True)\
    .drop_duplicates(subset="person_id",keep="first").index
    
    # First abnormal date 넣기
    abrnoaml_ctcae["first_abnormal_date"] = abrnoaml_ctcae.loc[abnormal_ctcae_idx]["measurement_date"]

    # target 의 person_id
    target_person=abrnoaml_ctcae["person_id"].unique()
    first_abnormal_date=abrnoaml_ctcae[abrnoaml_ctcae["first_abnormal_date"].notnull()].index
    #print("abnormal person:",len(target_person))   

    # abnormal 환자의 모든 레코드를 담기위해
    # abnormal person_id 수집
    abnormal_idx=[]
    for i in target_person:        
        abnormal_idx.extend(IO_measurement[IO_measurement["person_id"]==i].index)

    #abnormal dataframe
    abnormal_df=IO_measurement.loc[abnormal_idx]  
    abnormal_df["first_abnormal_date"]=abnormal_df.loc[first_abnormal_date]["measurement_date"]

    #해당 person 의 first_abnormal_date 채우기(그래야 row 별로 비교 가능)
    for person_id, group_data in abnormal_df.groupby(["person_id"]):    
        all_date=group_data.first_abnormal_date.unique()
        first_abnormal_date = [i for i in all_date][1]
        #print(first_abnormal_date)
        abnormal_df["first_abnormal_date"][abnormal_df["person_id"]==person_id]=first_abnormal_date

    # 전체 index 에서 abnormal index 제거해서 normal index 찾기
    normal_idx=IO_measurement.index.difference(abnormal_idx)
    normal_idx=normal_idx.tolist()
    normal_df=IO_measurement.loc[normal_idx] 

    # label 1,0 채우기
    abnormal_df["label"]=1
    normal_df["label"]=0
    print(len(abnormal_df["person_id"].unique())) # 
    print(len(normal_df["person_id"].unique()))   # 

    # reset_index
    abnormal_df=abnormal_df.sort_values(["person_id","cohort_start_date","measurement_date"]).reset_index(drop=True)
    normal_df=normal_df.sort_values(["person_id","cohort_start_date","measurement_date"]).reset_index(drop=True)

    # imputation
    abnormal_df["first_abnormal_date"]=abnormal_df["first_abnormal_date"].fillna('1970-01-01')
    abnormal_df["person_id"]=abnormal_df["person_id"].fillna(abnormal_df["subject_id"])
    abnormal_df["measurement_date"]=abnormal_df["measurement_date"].fillna(abnormal_df["cohort_start_date"])
    normal_df["person_id"]=normal_df["person_id"].fillna(normal_df["subject_id"])
    normal_df["measurement_date"]=normal_df["measurement_date"].fillna(normal_df["cohort_start_date"])

    abnormal_df=abnormal_df.fillna('0')
    normal_df=normal_df.fillna('0')
    
    dataset = pd.concat([abnormal_df,normal_df])
    dataset.sort_values(["person_id","cohort_start_date","measurement_date"]).reset_index(drop=True)
    
    return dataset

### hepatotoxicity (간독성)

In [ ]:
_3times = 3
_2times = 2

# 1) 간독성 
#    - 1-1) 신촌 Normal Range
#    - 1-2) 강남 Normal Range
hepatotoxicity_AST_pre=[['F',3013721,'>',34],  ['M',3013721,'>',34]]                    # AST : 13-34
hepatotoxicity_ALT_pre=[['F',3006926,'>',46],  ['M',3006926,'>',46]]                    # ALT : 5~46
hepatotoxicity_ALP_pre=[['F',3000963,'>',390], ['M',3000963,'>',151]]                   # ALP : male:50-151 female:50-390
hepatotoxicity_TBL_pre=[['F',3024128,'>',1.5], ['M',3024128,'>',1.8]]                   # TBL : male:0.2-1.8 female:0.2-1.5

hepatotoxicity_AST=[['F',3013721,'>',34*_3times],  ['M',3013721,'>',34*_3times]]        # AST : 13-34
hepatotoxicity_ALT=[['F',3006926,'>',46*_3times],  ['M',3006926,'>',46*_3times]]        # ALT : 5~46
hepatotoxicity_ALP=[['F',3000963,'>',390*_2times], ['M',3000963,'>',151*_2times]]       # ALP : male:50-151 female:50-390
hepatotoxicity_TBL=[['F',3024128,'>',1.5*_2times], ['M',3024128,'>',1.8*_2times]]       # TBL : male:0.2-1.8 female:0.2-1.5
#    - 1-3) 건양대 Normal Range

print('base dataset:',len(IO_measurement))

history_query_AST = """(gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} and value_as_number>{}) or \
    (gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} and value_as_number>{})""" \
    .format(hepatotoxicity_AST_pre[0][0],hepatotoxicity_AST_pre[0][1],hepatotoxicity_AST_pre[0][3], \
    hepatotoxicity_AST_pre[1][0],hepatotoxicity_AST_pre[1][1],hepatotoxicity_AST_pre[1][3])

history_query_ALT = """(gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} and value_as_number>{}) or \
    (gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} and value_as_number>{})""" \
    .format(hepatotoxicity_ALT_pre[0][0],hepatotoxicity_ALT_pre[0][1],hepatotoxicity_ALT_pre[0][3], \
    hepatotoxicity_ALT_pre[1][0],hepatotoxicity_ALT_pre[1][1],hepatotoxicity_ALT_pre[1][3])

history_query_ALP = """(gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} and value_as_number>{}) or \
    (gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} and value_as_number>{})""" \
    .format(hepatotoxicity_ALP_pre[0][0],hepatotoxicity_ALP_pre[0][1],hepatotoxicity_ALP_pre[0][3], \
    hepatotoxicity_ALP_pre[1][0],hepatotoxicity_ALP_pre[1][1],hepatotoxicity_ALP_pre[1][3])

history_query_TBL = """(gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} and value_as_number>{}) or \
    (gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} and value_as_number>{})""" \
    .format(hepatotoxicity_TBL_pre[0][0],hepatotoxicity_TBL_pre[0][1],hepatotoxicity_TBL_pre[0][3], \
    hepatotoxicity_TBL_pre[1][0],hepatotoxicity_TBL_pre[1][1],hepatotoxicity_TBL_pre[1][3])

IO_measurement_history_AST = IO_measurement.query(history_query_AST)
IO_measurement_history_ALT = IO_measurement.query(history_query_ALT)
IO_measurement_history_ALP = IO_measurement.query(history_query_ALP)
IO_measurement_history_TBL = IO_measurement.query(history_query_TBL)

history_person = set(IO_measurement_history_AST.subject_id.unique()) | \
                  set(IO_measurement_history_ALT.subject_id.unique()) | \
                  set(IO_measurement_history_ALP.subject_id.unique()) | \
                  set(IO_measurement_history_TBL.subject_id.unique())

IO_measurement_without_history = IO_measurement[~IO_measurement["subject_id"].isin(history_person)]
globals()['IO_measurement_hepatotoxicity'] = IO_measurement_without_history
print('after wash out:', len(IO_measurement_without_history), len(IO_measurement_without_history.subject_id.unique()))  

abnormal_query_AST = """(gender_source_value=='{}' and measurement_concept_id=={} and value_as_number>{}) or \
    (gender_source_value=='{}' and measurement_concept_id=={} and value_as_number>{})""" \
    .format(hepatotoxicity_AST[0][0],hepatotoxicity_AST[0][1],hepatotoxicity_AST[0][3], \
    hepatotoxicity_AST[1][0],hepatotoxicity_AST[1][1],hepatotoxicity_AST[1][3])

abnormal_query_ALT = """(gender_source_value=='{}' and measurement_concept_id=={} and value_as_number>{}) or \
    (gender_source_value=='{}' and measurement_concept_id=={} and value_as_number>{})""" \
    .format(hepatotoxicity_ALT[0][0],hepatotoxicity_ALT[0][1],hepatotoxicity_ALT[0][3], \
    hepatotoxicity_ALT[1][0],hepatotoxicity_ALT[1][1],hepatotoxicity_ALT[1][3])

abnormal_query_ALP = """(gender_source_value=='{}' and measurement_concept_id=={} and value_as_number>{}) or \
    (gender_source_value=='{}' and measurement_concept_id=={} and value_as_number>{})""" \
    .format(hepatotoxicity_ALP[0][0],hepatotoxicity_ALP[0][1],hepatotoxicity_ALP[0][3], \
    hepatotoxicity_ALP[1][0],hepatotoxicity_ALP[1][1],hepatotoxicity_ALP[1][3])

abnormal_query_TBL = """(gender_source_value=='{}' and measurement_concept_id=={} and value_as_number>{}) or \
    (gender_source_value=='{}' and measurement_concept_id=={} and value_as_number>{})""" \
    .format(hepatotoxicity_TBL[0][0],hepatotoxicity_TBL[0][1],hepatotoxicity_TBL[0][3], \
    hepatotoxicity_TBL[1][0],hepatotoxicity_TBL[1][1],hepatotoxicity_TBL[1][3])
    
abnormal_AST = IO_measurement_without_history.query(abnormal_query_AST)    
abnormal_ALT = IO_measurement_without_history.query(abnormal_query_ALT)    
abnormal_ALP = IO_measurement_without_history.query(abnormal_query_ALP)    
abnormal_TBL = IO_measurement_without_history.query(abnormal_query_TBL)

abnormal_person = set(abnormal_AST.subject_id.unique()) | \
                  set(abnormal_ALT.subject_id.unique()) | \
                  set(abnormal_ALP.subject_id.unique()) | \
                  set(abnormal_TBL.subject_id.unique())

abnormal_temp = IO_measurement_without_history[IO_measurement_without_history["subject_id"].isin(abnormal_person)]
abnormal_temp = pd.concat([abnormal_AST, abnormal_ALT, abnormal_ALP, abnormal_TBL])

globals()['abnormal_hepatotoxicity'] = abnormal_temp
print('after abnormal records :', len(abnormal_temp))  

### nephrotoxicity(신독성)

In [ ]:
_1_5times = 1.5
# 2) 신독성
#    - 2-1) 신촌 Normal Range
#    - 2-2) 강남 Normal Range
nephrotoxicity_creatinine_pre   =[['F',3016723,'>',0.91],  ['M',3016723,'>',1.19]]                              # creatinine : male:0.32-1.19 female:0.49-0.91
nephrotoxicity_creatinine       =[['F',3016723,'>',0.91*_1_5times],  ['M',3016723,'>',1.19*_1_5times]]          # creatinine : male:0.32-1.19 female:0.49-0.91
#    - 2-3) 건양대 Normal Range

print('base dataset:',len(IO_measurement), len(IO_measurement.subject_id.unique()))
history_query = """(gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} and value_as_number>{}) or \
    (gender_source_value=='{}' and cohort_start_date>=measurement_date and measurement_concept_id=={} and value_as_number>{})""" \
    .format(nephrotoxicity_creatinine_pre[0][0],nephrotoxicity_creatinine_pre[0][1],nephrotoxicity_creatinine_pre[0][3], \
    nephrotoxicity_creatinine_pre[1][0],nephrotoxicity_creatinine_pre[1][1],nephrotoxicity_creatinine_pre[1][3])
print(history_query)

IO_measurement_history = IO_measurement.query(history_query)
IO_measurement_without_history = IO_measurement[~IO_measurement["subject_id"].isin(IO_measurement_history.subject_id.unique())]
globals()['IO_measurement_nephrotoxicity'] = IO_measurement_without_history
print('after wash out:', len(IO_measurement_without_history), len(IO_measurement_without_history.subject_id.unique()))  

abnormal_query = """(gender_source_value=='{}' and measurement_concept_id=={} and value_as_number>{}) or \
    (gender_source_value=='{}' and measurement_concept_id=={} and value_as_number>{})""" \
    .format(nephrotoxicity_creatinine[0][0],nephrotoxicity_creatinine[0][1],nephrotoxicity_creatinine[0][3], \
    nephrotoxicity_creatinine[1][0],nephrotoxicity_creatinine[1][1],nephrotoxicity_creatinine[1][3])
print(abnormal_query)

IO_measurement_abnormal = IO_measurement_without_history.query(abnormal_query)    
globals()['abnormal_nephrotoxicity'] = IO_measurement_abnormal
print('after wash out:', len(IO_measurement_abnormal), len(IO_measurement_abnormal.subject_id.unique()))  

### outcome 마다 각 domain dataset 생성

In [ ]:
len(IO_measurement_hepatotoxicity.subject_id.unique())

In [ ]:
IO_measurement_abnormal=[IO_measurement_hepatotoxicity, IO_measurement_nephrotoxicity]
abnormal_list=[abnormal_hepatotoxicity, abnormal_nephrotoxicity]
abnormal_list_nm=['abnormal_hepatotoxicity', 'abnormal_nephrotoxicity']

# outcome 마다 각 domain dataset 생성
domains=['_measurement','_drug','_procedure','_condition']

#measurement
for IO_measurement, abnormal_ctcae, nm in zip(IO_measurement_abnormal, abnormal_list, abnormal_list_nm):
    dataset = measurement_targetting(abnormal_ctcae,IO_measurement)
    
    print(dataset)
    # population
    targetting = dataset[["subject_id","cohort_start_date","age","gender_source_value","first_abnormal_date","label"]]
    targetting=targetting.drop_duplicates()
    
    #measurement
    meas_1=dataset
    meas_1.rename(columns={'gender_source_value':'sex'},inplace=True)
#    globals()['{}{}'.format(nm,domains[0])] = meas_1
    meas_1.to_csv('{}/{}{}.txt'.format(output_dir,nm,domains[0]),index=False)
    
    #drug    
    drug_1=pd.merge(targetting, IO_drug, left_on=["subject_id"],right_on=["person_id"],how="inner")
    drug_1=drug_1.query('cohort_start_date_x <= drug_exposure_start_date')    
    drug_1=drug_1.rename(columns={'subject_id_x':'subject_id','cohort_start_date_x':'cohort_start_date',\
                                 'gender_source_value_x':'sex','person_id_x':'person_id',\
                                 'age_x':'age'},inplace=False)
#    globals()['{}{}'.format(nm,domains[1])] = drug_1.reset_index(drop=True)
    drug_1.to_csv('{}/{}{}.txt'.format(output_dir,nm,domains[1]),index=False)
    
    #procedure    
    proc_1=pd.merge(targetting, IO_procedure, left_on=["subject_id"],right_on=["person_id"],how="inner")
    proc_1=proc_1.query('cohort_start_date_x <= procedure_date')    
    proc_1=proc_1.rename(columns={'subject_id_x':'subject_id','cohort_start_date_x':'cohort_start_date',\
                                 'gender_source_value_x':'sex','person_id_x':'person_id',\
                                 'age_x':'age'},inplace=False)
#    globals()['{}{}'.format(nm,domains[2])] = proc_1.reset_index(drop=True)
    proc_1.to_csv('{}/{}{}.txt'.format(output_dir,nm,domains[2]),index=False)
    
    #condition    
    cond_1=pd.merge(targetting, IO_condition, left_on=["subject_id"],right_on=["person_id"],how="inner")
    cond_1=cond_1.query('cohort_start_date_x <= condition_start_date')    
    cond_1=cond_1.rename(columns={'subject_id_x':'subject_id','cohort_start_date_x':'cohort_start_date',\
                                 'gender_source_value_x':'sex','person_id_x':'person_id',\
                                 'age_x':'age'},inplace=False)
#   globals()['{}{}'.format(nm,domains[3])] = cond_1.reset_index(drop=True)  
    cond_1.to_csv('{}/{}{}.txt'.format(output_dir,nm,domains[3]),index=False) 


In [ ]:
meas_1.subject_id.unique()